### API Key 가져오기

In [6]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv(dotenv_path='.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_G


### 문제 1-1 기본 Chain 만들기 - AI 요리사

- 문제 설명 :
사용자가 재료를 입력하면 그 재료로 만들 수 있는 요리를 추천해주는 간단한 AI 요리사를 만들어보세요.

요구사항
1. PromptTemplate을 사용하여 프롬프트 작성
2. 사용자가 입력한 재료를 받아서 만들 수 있는 요리 추천
3. ChatOpenAI 모델 사용
4. StrOutputParser로 결과를 문자열로 출력
5. LCEL(|) 문법을 사용하여 체인 연결

- 구현 조건 : 
입력: 재료명 (예: "토마토, 양파, 치즈")
출력: 추천 요리와 간단한 레시피


In [7]:

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 컴포넌트 정의
prompt = PromptTemplate.from_template("당신은 요리사입니다.  <Question>: {input}를 이용한 추천요리와 간단한 레시피를 알려주세요")

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

# chain 연결 (LCEL)
chain = prompt | llm | StrOutputParser()

# 스트리밍 출력을 위한 요청
answer = chain.stream({"input": "토마토, 양파, 치즈"})
# 스트리밍 출력
#print(answer)

for token in answer:
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

토마토, 양파, 치즈를 이용한 추천요리인 '토마토, 양파 치즈 오믈렛'에 대해서 알려드리겠습니다.

**재료**

*   토마토 1개
*   양파 1/2개
*   치즈 1판
*   계란 2개
*   소금, 후추 

**레시피**

1.  양파를 깨끗이 씻은 후 채썰기를 합니다.
2.  토마토를 깨끗이 씻은 후, 네모로 썰어줍니다.
3.  계란을 깨어 소금과 후추로 간을 한 후, 잘 섞어줍니다.
4.  달군 프라이팬에 기름을 두르고, 양파와 토마토를 볶다가 토마토가 살짝 물러지면 접시에 덜어둡니다.
5.  같은 프라이팬에 기름을 두르고 계란물을 넣고 약불로 만들어서 계란이 반쯤 익으면, 볶아둔 토마토와 양파를 넣고, 치즈를 올립니다.
6.  팬의 가장자리가 익으면, 오믈렛을 반으로 접습니다.
7.  맛있는 '토마토, 양파 치즈 오믈렛'이 완성되었습니다.

### 문제 1-2 Multi Chain 만들기 - 여행지 정보 시스템
- 문제 설명 :
    사용자가 특정 도시나 국가를 입력하면, 해당 지역의 대표적인 관광 명소를 추천하고 그 명소에 대한 상세 정보(역사, 특징, 방문 팁 등)를 알려주는 2단계 체인을 구현해 보세요.
- 요구사항
1. 1단계 체인: 여행지(도시/국가)를 입력받아 대표 명소 1가지 추천
2. 2단계 체인: 추천받은 명소의 상세 정보 제공
3. ChatPromptTemplate 사용 
: “system”  과 “user” 메시지를 지정합니다.
4. 두 체인을 LCEL로 연결
5. 각 단계의 결과를 모두 출력하여 과정 확인
- 구현 조건
    - 입력: 여행지 (예: "로마", "뉴욕", "도쿄")
    - 1단계 출력: 추천 명소 이름
    - 2단계 출력: 명소에 대한 상세 정보 (역사, 특징, 방문 팁 등)



In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from pprint import pprint


prompt1 = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 여행유튜버입니다.") , 
     ("user", "{place} 에서 대표명소 한 곳을 추천해주세요") ]
)
prompt2 = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 여행유튜버입니다.") , 
     ("user", "{landmark}가 어딘지 알려주고 이 곳에 대한 상세정보를 알려주세요") ]
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1", 
    model="moonshotai/kimi-k2-instruct-0905", 
    temperature=0.7
)

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    {"landmark": chain1}  #
    | prompt2
    | llm
    | StrOutputParser()
)

response = chain2.invoke({"place": "프랑스"})
print("\n🔹 명소 소개:")
pprint(response)

NameError: name 'OPENAI_API_KEY' is not defined

### 문제1-3  FewShotPromptTemplate과 시스템 메시지 활용 

- 뉴스 키워드 추출기
1) 문제 설명
    - FewShotPromptTemplate을 사용하여 뉴스 기사에서 핵심 키워드 3개를 추출하는 시스템을 구현해보세요. 주어진 예시들을 참고하여 일관된 형식으로 키워드를 추출해야 합니다.
2) 요구사항
    - FewShotPromptTemplate 사용
    - 최소 3개의 예시(examples) 포함
    - 뉴스 텍스트에서 핵심 키워드 3개 추출
    - 일관된 출력 형식 유지
    - 다양한 분야의 뉴스로 테스트


In [ ]:
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate, 
    ChatPromptTemplate
)

# 1. 예시 프롬프트 (대화형)
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])


examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다.\
        이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. \
        전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    # 추가 예시 필요
]



# 2. Few-Shot 프롬프트
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 3. 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요."),
    few_shot_prompt,
    ("human", "{input}")
])

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input":"제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 \
                       버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며,\
                        텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='뉴스 키워드 추출 전문가입니다. 핵심 키워드 3개를 추출하세요.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'news': '삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.', 'keywords': '삼성전자, 인공지능, 엔비디아'}, {'news': '세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.', 'keywords': '세계보건기구, 건강위기, 국제협력'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['keywords', 'news'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['news'], input_types={}, partial_variables={}, template='{news}'), additional_kwargs={}), AIMess